# Multi - Period Binomial Model for Option Pricing


This workbook looks at multi period binomiaal option pricing model. Here the binomial tree is considered as a network with node (i, j) where i is the time step and j is the level of the tree. For example two period binomial tree will have i = {0, 1, 2} time steps and j = {0, 1, 2, 3} levels. 

To price the option we apply the following formula recurrsively backwards. 

* Risk neutral probability formula for one period binomiial model
$$\tilde{p} = \frac{e^{rT}-d}{u-d}$$ 

* Delta-hedging formula
$$\delta = \frac{C_u-C_d}{S_u-S_d}$$

* Derivative security that pays V_N at time N should be priced at 
$$C_{N-1} = 
e^{-rT}(\tilde{p} C_u^N +(1-\tilde{p})C_d^N)$$ at time $N-1$

where 

$r$ = annual risk-free interest rate, $T$ = maturity time, $S_0$ = initial stock price, $C_u$ and $C_d$ are the up and down payoff values at the time of maturity. 

## Binomial Tree Representation

Stock tree can be represented using nodes $(i,j)$ and intial stock price $S_0$

$S_{i,j} = S_0u^{j}d^{i-j}$

$C_{i,j}$ represents contract price at each node $(i,j)$. Where $C_{N,j}$ represents final payoff function that we can define based on the option types considered.

For this work we will price a
1. European call option, so $C_{N,j} = max(S_{N,j}-K,0)$ and
2. European put option, so $C_{N,j} = max(K-S_{N,j},0)$

The next section will describe how to implement the model to price American call and put options. The difference between American and the European options is that while European options are exercised only at maturity, American options can be exercised at any time during the time period. 




In [8]:
import numpy as np
import math

In [46]:
#pay off functions for European Call option
def pay_off_call(S, K, N):
    C = np.maximum( S - K , np.zeros(N+1) )
    return C

#pay off functions for European Put option
def pay_off_put(S, K, N):
    C = np.maximum( K-S , np.zeros(N+1) )
    return C

In [47]:
# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06     # annual risk-free rate
N = 3        # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
#opttype = 'C' # Option Type 'C' or 'P'

# European call option pricing

In [48]:
def binomial_tree_fast(K,T,S0,r,N,u,d):   
    #precompute constants
    dt = T/N
    p = (math.exp(r*dt)-d)/(u-d) 
    disc = math.exp(-r*dt)

    # initialise asset prices at maturity - Time step N
    S = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))

    # initialise option values at maturity
    C = pay_off_call(S, K, N)

    # step backwards through tree
    for i in np.arange(N,0,-1):
        C = disc * ( p * C[1:i+1] + (1-p) * C[0:i] )

    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d)   

10.145735799928826

# European put option pricing

In [49]:
def binomial_tree_fast(K,T,S0,r,N,u,d):   
    #precompute constants
    dt = T/N
    p = (math.exp(r*dt)-d)/(u-d) 
    disc = math.exp(-r*dt)
    
    # initialise asset prices at maturity - Time step N
    S = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))

    # initialise option values at maturity
    C = pay_off_put(S, K, N)

    # step backwards through tree
    for i in np.arange(N,0,-1):
        C = disc * ( p * C[1:i+1] + (1-p) * C[0:i] )

    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d)  

4.322189158353709

# American call option pricing

In [50]:
def binomial_tree_fast(K,T,S0,r,N,u,d):   
    #precompute constants
    dt = T/N
    p = (math.exp(r*dt)-d)/(u-d) 
    disc = math.exp(-r*dt)
    
    # initialise asset prices at maturity - Time step N
    S = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))
    
    # initialise option values at maturity
    C = pay_off_call(S, K, N)
    
    # step backwards through tree
    for i in np.arange(N,0,-1):
        for j in range(i):
            C[j] = disc * ( p * C[j+1] + (1-p) * C[j] )
            S[j] = S[j]/d
            C[j] = max(C[j], K-S[j])
    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d) 

13.059135847285697

# American put option pricing

In [51]:
def binomial_tree_fast(K,T,S0,r,N,u,d):   
    #precompute constants
    dt = T/N
    p = (math.exp(r*dt)-d)/(u-d) 
    disc = math.exp(-r*dt)
    
    # initialise asset prices at maturity - Time step N
    S = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))
    
    # initialise option values at maturity
    C = pay_off_Euro_put(S, K, N)
    
    # step backwards through tree
    for i in np.arange(N,0,-1):
        for j in range(i):
            C[j] = disc * ( p * C[j+1] + (1-p) * C[j] )
            S[j] = S[j]/d
            C[j] = max(C[j], K-S[j])
    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d) 

4.654588754602527

# Reference

"Implementing Derivatives Models" by Les Clewlow and Chris Strickland